In [1]:
!pip install -q langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="<API_KEY>"
os.environ["LANGCHAIN_PROJECT"]="RAG_ADVANCED"
os.environ['OPENAI_API_KEY'] = "<API_KEY>"


from langsmith import utils
utils.tracing_is_enabled()

True

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [4]:
#Load Documents
loader = WebBaseLoader(
    web_paths = ("https://lilianweng.github.io/posts/2023-06-23-agent/", ),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [9]:
#Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap=50)
splits = text_splitter.split_documents(docs)

#Embed
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

vectorestore = Chroma.from_documents(documents=splits, embedding = embedding, persist_directory="./db006")

#Retriever
retriever = vectorestore.as_retriever()

In [8]:
from langchain.prompts import ChatPromptTemplate

#Decomposition
template = """You are a helful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub questions that can be answers in isolation. \n
Generate multiple search queries related to:{question}\n
Output (3 queries):"""

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [10]:
#LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#Chain
generate_queries_decomposition = (prompt_decomposition | llm | StrOutputParser() | (lambda x:x.split("\n")))

#Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke(question)

In [14]:
#RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
  """RAG on each sub-question"""

  #Use our decomposition
  sub_questions = sub_question_generator_chain.invoke({"question":question})

  #Initialize a list to hold RAG chain results
  rag_results = []

  for sub_question in sub_questions:
    #Retrieve documents for each sub_question
    retrieved_docs = retriever.invoke(sub_question)

    #Use retrieved documents and sub_question in RAG chain
    answer = (prompt_rag | llm | StrOutputParser()).invoke({"context":retrieved_docs, "question":sub_question})

    rag_results.append(answer)

  return rag_results, sub_questions

answers, question = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)


In [15]:
def format_qa_pairs(question, answer):
  """Format Q and A pairs"""
  formatted_string = ""
  for i, (question, answer) in enumerate(zip(question, answer), start =1):
    formatted_string+= f"Question {i}: {question}\nAnswer {i}:{answer}\n\n"
  return formatted_string.strip()

context = format_qa_pairs(questions, answers)

#prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)


final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context, "question":question})

"The essential architectural elements of large language models (LLMs) utilized in autonomous agent systems include the LLM itself as the core controller, a natural language interface for communication with external components, and additional components such as memory and tools. These elements work together to enable the agent to perform tasks effectively, although the reliability of the natural language interface can be a concern due to potential errors in model outputs.\n\nRegarding the contribution of different types of sensors to perception in LLM-powered autonomous agents, the retrieved context does not provide specific information on this aspect, so it remains unclear how sensors integrate into the system's perception capabilities.\n\nFor decision-making and planning in LLM-based autonomous systems, common algorithms include classical planners that utilize the Planning Domain Definition Language (PDDL) for long-horizon planning. The LLM acts as the brain of the system, parsing use